In [5]:
import sys
import os
import yaml
import configparser

# Add the src directory to the sys.path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

Consideraciones:
* generar schema de validacion de streams
* introducir crdenciales a traves del entorno de docker - ENV variables en .ipynb no funcionan
* optimizar filtrado de id incluir particion?
* structura de micro servicios: inventory consumer - producer
* diseño de sistemas: filtrar ordenes efectuadas para los siguientes envios
* confirmar Best pracitces para el manejo de streams: memorytable vs readStream
* confirmar best practice para pertion key del stream
* Arquitectura Lambda
* builling de S3 por fichero registrado: definir criterios de particion
* emplear memory format para lectura de streams
* best practices para gestionar checkpoints
* firehose implementarlo

# 1. Environment Configuration

## 1.1 Import dependencies

In [2]:
import boto3
import json
import os
from uuid import uuid4
from datetime import datetime
from datetime import timedelta
import time
import random
import uuid
import logging
import numpy as np

import pyspark.sql.types as t
import pyspark.sql.functions as f

In [3]:
from spark_session import create_spark_session
from schemas import *
from functions import *

## 1.2 Extract AWS credentials

In [6]:
def load_aws_credentials(profile_name="default"):

    # Load credentials from the .aws/credentials file (local development)
    try:
        credentials = configparser.ConfigParser()
        credentials.read(os.path.join('..', '.aws', 'credentials'))
        
        logging.info("Successfully loaded credentials variables from .aws file.")
    except Exception as e:
        logging.error(f"Error loading .aws file: {e}")
        sys.exit(1)

    aws_access_key_id = credentials[profile_name]["aws_access_key_id"]
    aws_secret_access_key = credentials[profile_name]["aws_secret_access_key"]

    if not aws_access_key_id or not aws_secret_access_key:
        logging.error("AWS credentials not found.")
        sys.exit(1)

    return aws_access_key_id, aws_secret_access_key

aws_access_key_id, aws_secret_access_key = load_aws_credentials()

## 1.3 Constants variables

In [5]:
# Initialize logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [10]:
def load_aws_config():
    """
    Loads AWS configuration settings from the .aws/config file.

    :param profile_name: The profile name in the AWS config file (default: "default").
    :return: The region_name as a string.
    """
    try:
        config = configparser.ConfigParser()
        config.read(os.path.join('..', '.aws', 'config'))
        logging.info("Successfully loaded config variables from .aws file.")

        return config
    except Exception as e:
        logging.error(f"Error loading .aws file: {e}")
        sys.exit(1)

config = load_aws_config()

In [12]:
BUCKET_NAME = config["paths"]["BUCKET_NAME"]
RAW = config["paths"]["RAW"]
ORDERS = config["paths"]["ORDERS"]

BRONZE = config["paths"]["BRONZE"]
SILVER = config["paths"]["SILVER"]
GOLD = config["paths"]["GOLD"]

ADDRESS_DATA = config["raw_data"]["ADDRESS_DATA"]
CLIENTS_DATA = config["raw_data"]["CLIENTS_DATA"]
PRODUCTS_DATA = config["raw_data"]["PRODUCTS_DATA"]

ADDRESS_TABLE = config["table_names"]["ADDRESS_TABLE"]
CLIENTS_TABLE = config["table_names"]["CLIENTS_TABLE"]
CLIENTS_ADDRESS_TABLE = config["table_names"]["CLIENTS_ADDRESS_TABLE"]
PRODUCTS_TABLE = config["table_names"]["PRODUCTS_TABLE"]
PACKAGE_TABLE = config["table_names"]["PACKAGE_TABLE"]

RAW_ADDRESS_PATH = os.path.join(BUCKET_NAME, RAW, ADDRESS_DATA)
RAW_CIENTS_PATH = os.path.join(BUCKET_NAME, RAW, CLIENTS_DATA)
RAW_PRODUCTS_PATH = os.path.join(BUCKET_NAME, RAW, PRODUCTS_DATA)

BRONZE_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, ADDRESS_TABLE)
BRONZE_CLIENTS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, CLIENTS_TABLE)
BRONZE_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, PRODUCTS_TABLE)


SILVER_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, ADDRESS_TABLE)
SILVER_CLIENTS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, CLIENTS_TABLE)
SILVER_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, PRODUCTS_TABLE)

GOLD_CLIENTS_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, CLIENTS_ADDRESS_TABLE)
GOLD_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, PRODUCTS_TABLE)
GOLD_PACKAGE_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, PACKAGE_TABLE)

In [14]:
stream_name = config["default"]["STREAM_NAME"]

# 2. Initialize Spark Session

In [15]:
spark = create_spark_session(aws_access_key_id, aws_secret_access_key)

24/09/22 00:14:27 WARN Utils: Your hostname, Miguels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.16 instead (on interface en0)
24/09/22 00:14:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/miguelgranica/Documents/MBIT-DE/vpr-data_publisher/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/miguelgranica/.ivy2/cache
The jars for the packages stored in: /Users/miguelgranica/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-03f19d38-7c39-42b2-a0d5-b88a7f84a7e5;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 143ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	

# 2. Data generation

In [16]:
def generate_order_payload(order_details):
    """
    Generate a payload for an order event.

    :param order_details: Dictionary containing order details.
    :return: Dictionary containing the payload for the order event.
    """
    return {
        "event_id": f"ev-{uuid.uuid4()}",
        "event_type": "ORDER_CREATED",
        "event_timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "order_id": f"ord-{uuid.uuid4()}",
        "order_details": order_details
    }

In [17]:
def generate_order_details(df_clients, df_products, df_packages):
    """
    Generate order details based on client information and item list.

    :param df_clients: DataFrame containing client information.
    :param df_products: DataFrame containing product information.
    :param df_packages: DataFrame containing package information.
    :return: Dictionary containing order details.
    """
    current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    item_list = generate_item_list(df_products, df_packages)
    client_details = select_client_order_details(df_clients)

    return {
        "customer_id": client_details["client_id"],
        "order_timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "order_date": datetime.now().strftime('%Y-%m-%d'),
        "items": item_list,
        "total_amount": generate_item_agg(item_list, "price"),
        "total_volume": generate_item_measure_agg(item_list, "volume"),
        "status": "RECEIVED",
        "destination_address": generate_destination_address_dict(client_details),
        "payment_details": {
            "payment_method": "",
            "payment_status": "",
            "transaction_id": ""
        }
    }

def select_client_order_details(df, primary_key_col="address_id"):
    """
    Select a random row from a DataFrame based on a unique primary key column.
    
    :param df: DataFrame to select from.
    :param primary_key_col: Name of the primary key column. default value: address_id
    :return: DataFrame containing a single randomly selected row.
    """
    # Get a list of all primary key values
    primary_keys = df.select(primary_key_col).rdd.flatMap(lambda x: x).collect()

    # Randomly select one primary key value
    random_primary_key = random.choice(primary_keys)

    # Filter the DataFrame to get the row with the random primary key
    random_row_df = df.filter(f.col(primary_key_col) == random_primary_key)

    # Convert the DataFrame row to dictionary and return
    return random_row_df.first().asDict() if random_row_df else None

def select_product_order_details(df_product, df_package,  quantity=3, primary_key_col="product_id"):
    """
    Select a random row from a DataFrame based on a unique primary key column.
    
    :param df: DataFrame to select from.
    :param primary_key_col: Name of the primary key column. default value: product_id
    :param quantity: Maximum quantity of each product (default: 3).
    :return: DataFrame containing a single randomly selected row.
    """
    # Get a list of all primary key values
    primary_keys = df_product.select(primary_key_col).rdd.flatMap(lambda x: x).collect()

    # Randomly select one primary key value
    random_primary_key = random.choice(primary_keys)

    # Register random choice function as a UDF
    weighted_random_choice_udf = f.udf(weighted_random_choice, t.IntegerType())

    # Filter the DataFrame to get the row with the random primary key
    random_product_df = (
        df_product
        .filter(f.col(primary_key_col) == random_primary_key)
        # Set random product quantity
        .withColumn("product_quantity", weighted_random_choice_udf(f.lit(quantity)))
        # Explode package annidations
        .withColumn("product_components_explode", f.explode(f.col("product_components")))
        # Extract package information
        .withColumn("package_id", f.explode(f.col("product_components_explode.package_id")))
        .withColumn("package_quantity", f.explode(f.col("product_components_explode.package_quantity")))
        # Rename column
        .withColumnRenamed("name", "product_name")
        # Join package measures
        .join(
            df_package, on="package_id", how="left"
        )
        # Select Columns
        .select(
            f.col("product_id"),
            f.col("product_quantity"),
            f.col("package_id"),
            f.col("product_name"),
            f.col("price"),
            f.col("package_quantity"),
            f.col("width"),
            f.col("height"),
            f.col("length"),
            f.col("volume")
        )
    )

    # Convert each DataFrame row to list of dictionary and return
    return [row.asDict() for row in random_product_df.collect()]

def generate_destination_address_dict(clients_dict):
    """
    Filter unnecessary keys from the client's address dictionary.

    :param clients_dict: Dictionary containing client information.
    :return: Dictionary containing filtered address information.
    """
    address_keys = [
        'address_id', 'neighborhood', 'coordinates', 'road', 'house_number',
        'suburb', 'city_district', 'state', 'postcode', 'country', 'lat', 'lon'
    ]
    return {k: v for k, v in clients_dict.items() if k in address_keys}

def generate_item_list(df_products, df_packages, items=5, quantity=3):
    """
    Generate a list of items with details.
    
    Parameters:
    - items: Number of items to generate details for (default: 5).
    - quantity: Maximum quantity of each item (default: 3).

    Returns:
    - List of dictionaries containing item details.
    """
    return [        
        {
            "product_id": packages[0]["product_id"], 
            "product_name": packages[0]["product_name"], 
            "price": packages[0]["price"],
            "quantity": packages[0]["product_quantity"],
            "packages": [
                {
                    "package_id": package["package_id"],
                    "quantity": package["package_quantity"],
                    "volume": package["volume"]
                } for package in packages 
            ],
        }
        for packages in [select_product_order_details(df_products, df_packages) for num in range(weighted_random_choice(5))]
    ]
    
    # return [ 
    #     package 
    #     for num in range(weighted_random_choice(5)) 
    #     for package in select_product_order_details(df_products, df_packages, quantity)
    # ]

    # return [
    #     {
    #         "product_id": item["product_id"], 
    #         "product_name": item["product_name"], 
    #         "price": item["price"], 
    #         "weight": item["weight"],
    #         "quantity": weighted_random_choice(quantity)
    #     }
    #     for item in [ select_client_order_details(df_products, primary_key_col="product_id") for num in range(weighted_random_choice(items))]
    # ]

def weighted_random_choice(numbers_len):
    """
    Select a random number from a range starting from 1 with weights based on reciprocal values.

    Parameters:
    - numbers_len: Length of the range of numbers starting from 1.

    Returns:
    - A randomly selected number based on the reciprocal weights.
    """
    # Define numbers range starting from 1 to numbers_len
    numbers = np.arange(1, numbers_len + 1)
    
    # Calculate weights based on reciprocal values
    weights = 1 / numbers
    
    # Ensure the weights sum to 1
    normalized_weights = weights / np.sum(weights)
    
    # Select a random number with the specified weights
    random_number = int(np.random.choice(numbers, p=normalized_weights))
    
    return random_number

def generate_item_agg(items, property_name):
    """
    Generate the aggregate value of a property for a list of items.

    :param items: List of dictionaries containing item details.
    :param property_name: Name of the property to aggregate.
    :return: Aggregate value of the specified property.
    """
    return sum([(item['quantity'] * item[property_name]) for item in items])

def generate_item_measure_agg(items, property_name="volume", quantity="quantity"):
    """
    Generate the aggregate value of a specified property for a list of items.

    This function calculates the sum of the product of item quantities, package quantities, 
    and a specified property (e.g., volume) for each package within each item. 
    If the specified property is `None`, it is treated as `1` to ensure the multiplication proceeds.

    :param items: List of dictionaries, where each dictionary contains details about an item and its packages.
    :param quantity: The key name in the dictionaries for the quantity of the item and packages (default is "quantity").
    :param property_name: The key name in the dictionaries for the property to aggregate (default is "volume").
    :return: The aggregated value of the specified property.
    """
    return sum([
        item[quantity]* package[quantity] * (package[property_name] if package[property_name] is not None else 1) 
        for item in items 
        for package in item['packages']
    ])

In [18]:
df_clients_address = read_file(spark, GOLD_CLIENTS_ADDRESS_PATH, "parquet", gold_clients_address_schema)
df_products = read_file(spark, GOLD_PRODUCTS_PATH, "delta")#spark.read.format("delta").load(GOLD_PRODUCTS_PATH)
df_packages = read_file(spark, GOLD_PACKAGE_PATH, "delta") #spark.read.format("delta").load(GOLD_PACKAGE_PATH)

24/09/22 00:20:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [26]:
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- url: string (nullable = true)
 |-- price: float (nullable = true)
 |-- currency: string (nullable = true)
 |-- product_components: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- package_id: string (nullable = true)
 |    |    |-- subpackage_id: integer (nullable = true)
 |    |    |-- package_quantity: integer (nullable = true)



In [20]:
df_packages.show()

+----------+-------------+----------+-----+------+------+------+-------+--------------+
|package_id|subpackage_id|      name|width|height|length|weight| volume|stock_quantity|
+----------+-------------+----------+-----+------+------+------+-------+--------------+
|  00245842|            1|     BESTÅ| 41.0|   8.0| 196.0| 18.01|64288.0|          1000|
|  00263850|            1|     BILLY| 29.0|  13.0| 206.0|  37.9|77662.0|          1000|
|  00275848|            1|    KALLAX| 41.0|  12.0| 155.0|  15.2|76260.0|          1000|
|  00278578|            1|    HYLLIS| 29.0|   4.0| 140.0|  5.76|16240.0|          1000|
|  00286677|            1|     HEJNE|  7.0|   3.0| 171.0|  1.77| 3591.0|          1000|
|  00295554|            1|     BESTÅ| 36.0|   2.0|  59.0|  2.47| 4248.0|          1000|
|  00324518|            1|    KALLAX| 41.0|  16.0| 150.0| 21.14|98400.0|          1000|
|  00340047|            1|      EKET|  9.0|   3.0|  33.0|  0.32|  891.0|          1000|
|  00415603|            1|     B

In [36]:
# Register the function as a UDF
weighted_random_choice_udf = f.udf(weighted_random_choice, t.IntegerType())
(
    df_products
    # Set random product quantity
    .withColumn("product_quantity", weighted_random_choice_udf(f.lit(3)))
    # Explode package annidations
    .withColumn("product_components_explode", f.explode(f.col("product_components")))
    # Extract package information
    .withColumn("package_id", f.col("product_components_explode.package_id"))
    .withColumn("subpackage_id", f.col("product_components_explode.subpackage_id"))
    .withColumn("package_quantity", f.col("product_components_explode.package_quantity"))
    .withColumnRenamed("name", "product_name")
    .join(
        df_packages, on=["package_id", "subpackage_id"], how="left"
    )
    # Select Columns
    .select(
        f.col("product_id"),
        f.col("product_quantity"),
        f.col("package_id"),
        f.col("subpackage_id"),
        # f.col("name"),
        f.col("product_name"),
        f.col("price"),
        f.col("package_quantity"),
        f.col("width"),
        f.col("height"),
        f.col("length"),
        f.col("weight"),
        f.col("volume")
    )
).show()

+--------------------+----------------+----------+-------------+------------+------+----------------+-----+------+------+------+-------+
|          product_id|product_quantity|package_id|subpackage_id|product_name| price|package_quantity|width|height|length|weight| volume|
+--------------------+----------------+----------+-------------+------------+------+----------------+-----+------+------+------+-------+
|prod-00bea5f6-788...|               1|  00415603|            1|       BILLY| 169.0|               1| 29.0|  13.0| 205.0| 34.79|77285.0|
|prod-00bea5f6-788...|               1|  00415603|            2|       BILLY| 169.0|               1| 40.0|   5.0| 195.0| 14.27|39000.0|
|prod-00e31374-ddc...|               2|  10261129|            1|      FINNBY| 49.99|               1| 27.0|   7.0| 182.0|  19.2|34398.0|
|prod-01054ca9-a8a...|               2|  50339292|            1|     FJÄLLBO| 99.99|               1| 42.0|  16.0| 104.0| 17.64|69888.0|
|prod-023e6bbb-bcd...|               1|  

24/09/22 02:10:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1055985 ms exceeds timeout 120000 ms
24/09/22 02:10:53 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/22 02:10:54 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [24]:
# [        
#     {
#         "product_id": packages[0]["product_id"], 
#         "product_name": packages[0]["product_name"], 
#         "price": packages[0]["price"],
#         "quantity": packages[0]["product_quantity"]
#         "packages": [
#             {
#                 "package_id": package["package_id"],
#                 "quantity": package["package_quantity"],
#                 "volume": package["volume"]
#             } for package in packages 
#         ],
#     }
#     for packages in [select_product_order_details(df_products, df_packages) for num in range(weighted_random_choice(5))]
# ]

select_product_order_details(df_products, df_packages)

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "explode(product_components_explode.package_id)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "product_components_explode.package_id" has the type "STRING".;
'Project [product_id#94, name#95, category#96, url#97, price#98, currency#99, product_components#100, product_quantity#2260, product_components_explode#2270, explode(product_components_explode#2270.package_id) AS package_id#2280]
+- Project [product_id#94, name#95, category#96, url#97, price#98, currency#99, product_components#100, product_quantity#2260, product_components_explode#2270]
   +- Generate explode(product_components#100), false, [product_components_explode#2270]
      +- Project [product_id#94, name#95, category#96, url#97, price#98, currency#99, product_components#100, weighted_random_choice(3)#2259 AS product_quantity#2260]
         +- Filter (product_id#94 = prod-4166d7a9-5dc6-4e09-b929-a9dcbb88eb35)
 

In [23]:
item_list = generate_item_list(df_products, df_packages)
item_list

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "explode(product_components_explode.package_id)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "product_components_explode.package_id" has the type "STRING".;
'Project [product_id#94, name#95, category#96, url#97, price#98, currency#99, product_components#100, product_quantity#2130, product_components_explode#2140, explode(product_components_explode#2140.package_id) AS package_id#2150]
+- Project [product_id#94, name#95, category#96, url#97, price#98, currency#99, product_components#100, product_quantity#2130, product_components_explode#2140]
   +- Generate explode(product_components#100), false, [product_components_explode#2140]
      +- Project [product_id#94, name#95, category#96, url#97, price#98, currency#99, product_components#100, weighted_random_choice(3)#2129 AS product_quantity#2130]
         +- Filter (product_id#94 = prod-f7b5aac4-179c-47b0-b0b9-7fb2ebc31248)
 

In [18]:
generate_item_measure_agg(item_list)

130851

In [19]:
order_details = generate_order_details(df_clients_address, df_products, df_packages)
print(json.dumps(order_details, indent=4))

{
    "customer_id": "cus-1186ed3d-e052-4636-8dc9-4814bf728151",
    "order_timestamp": "2024-09-11 11:05:34",
    "order_date": "2024-09-11",
    "items": [
        {
            "product_id": "prod-d76cb769-cccc-4ac1-a730-2acdf3389bda",
            "product_name": "BILLY / OXBERG",
            "price": 139.99000549316406,
            "quantity": 3,
            "packages": [
                {
                    "package_id": "69281776",
                    "quantity": 1,
                    "volume": null
                },
                {
                    "package_id": "00263850",
                    "quantity": 1,
                    "volume": 77662
                },
                {
                    "package_id": "50275558",
                    "quantity": 2,
                    "volume": 25320
                }
            ]
        }
    ],
    "total_amount": 419.9700164794922,
    "total_volume": 384909,
    "status": "RECEIVED",
    "destination_address": {
        

In [20]:
order_stream = generate_order_payload(order_details)
print(json.dumps(order_stream, indent=4))

{
    "event_id": "ev-ee459083-b647-457e-aa5a-2ae01701e66a",
    "event_type": "ORDER_CREATED",
    "event_timestamp": "2024-09-11 11:06:09",
    "order_id": "ord-ec7a5787-1ab0-462c-935b-064a0950b206",
    "order_details": {
        "customer_id": "cus-1186ed3d-e052-4636-8dc9-4814bf728151",
        "order_timestamp": "2024-09-11 11:05:34",
        "order_date": "2024-09-11",
        "items": [
            {
                "product_id": "prod-d76cb769-cccc-4ac1-a730-2acdf3389bda",
                "product_name": "BILLY / OXBERG",
                "price": 139.99000549316406,
                "quantity": 3,
                "packages": [
                    {
                        "package_id": "69281776",
                        "quantity": 1,
                        "volume": null
                    },
                    {
                        "package_id": "00263850",
                        "quantity": 1,
                        "volume": 77662
                    },
           

In [21]:
order_stream['order_id'] #['orderDetails']['destinationAddress']['neighborhood']

'ord-ec7a5787-1ab0-462c-935b-064a0950b206'

# 3. Stream producer

In [31]:
kinesis_client = boto3.client(
    'kinesis', 
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    # aws_session_token=aws_session_token,
    region_name='eu-south-2'
)

In [32]:
def produce_order(payload):
    try:
        # Ensure payload is correctly formatted and partition key is a string
        if 'event_type' not in payload or not isinstance(payload['event_type'], str):
            raise ValueError("Payload must include 'event_type' as a string")
        
        data = json.dumps(payload)
        put_response = kinesis_client.put_record(
            StreamName=stream_name,
            Data=f"{data}\n",
            PartitionKey=payload['event_type']
        )
        
        # Log response details
        logger.info(f"Put record response: {put_response}")
        return put_response
    except Exception as e:
        logger.error(f"Failed to put record to stream: {e}", exc_info=True)
        return None

In [33]:
order_payload = generate_order_payload(generate_order_details(df_clients_address, df_products, df_packages))
print(json.dumps(order_payload, indent=4))

{
    "event_id": "ev-349f72a5-1a9d-4daf-b054-eff84821ee8a",
    "event_type": "ORDER_CREATED",
    "event_timestamp": "2024-09-11 11:22:59",
    "order_id": "ord-1df95ddd-b917-45cb-b786-b51cef48206a",
    "order_details": {
        "customer_id": "cus-1a44f64e-9b47-45a3-a77c-707ac113d4ba",
        "order_timestamp": "2024-09-11 11:22:59",
        "order_date": "2024-09-11",
        "items": [
            {
                "product_id": "prod-f90a34ca-44da-4075-a14b-686919fa799c",
                "product_name": "BILLY",
                "price": 139.97000122070312,
                "quantity": 1,
                "packages": [
                    {
                        "package_id": "99395936",
                        "quantity": 1,
                        "volume": null
                    },
                    {
                        "package_id": "50263838",
                        "quantity": 3,
                        "volume": 43470
                    },
                    

In [34]:
order_payload = generate_order_payload(generate_order_details(df_clients_address, df_products, df_packages))
produce_order(order_payload)

INFO:root:Put record response: {'ShardId': 'shardId-000000000001', 'SequenceNumber': '49655731157571739728119313308661653694831254950926876690', 'ResponseMetadata': {'RequestId': 'e5dc422a-abc3-a339-ba87-2190005f8ae7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e5dc422a-abc3-a339-ba87-2190005f8ae7', 'x-amz-id-2': 'dRj7nirHcP8dJ7kUebkNCKw9YQ1qG7uzqrntGkr7bWlfDkVgLDO5B9LrxBXo0gz53zpNceWsRtHh4+GfGskyWUad3XjB5U86', 'date': 'Wed, 11 Sep 2024 09:23:06 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


{'ShardId': 'shardId-000000000001',
 'SequenceNumber': '49655731157571739728119313308661653694831254950926876690',
 'ResponseMetadata': {'RequestId': 'e5dc422a-abc3-a339-ba87-2190005f8ae7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e5dc422a-abc3-a339-ba87-2190005f8ae7',
   'x-amz-id-2': 'dRj7nirHcP8dJ7kUebkNCKw9YQ1qG7uzqrntGkr7bWlfDkVgLDO5B9LrxBXo0gz53zpNceWsRtHh4+GfGskyWUad3XjB5U86',
   'date': 'Wed, 11 Sep 2024 09:23:06 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [ ]:
# Produce orders at regular intervals (for example, every second)
# while True:
#     order_payload = generate_order_payload(generate_order_details(df_clients_address, df_products))
#     produce_order(order_payload)
#     time.sleep(20)